In [1]:
import pandas as pd
import warnings

In [2]:
warnings.filterwarnings("ignore")

### Carregando o dataset

In [3]:
df = pd.read_csv('data/goodreads.csv', sep=';')

### Selecionando as isntâncias úteis

A coluna `is_read` presente no dataset é uma coluna que indica se o livro foi ou não lido pelo usuário, portanto instâncias na qual o valor dessa coluna é 'false' serão inúteis, visto que se um usuário não leu um livro ele não terá avaliado o mesmo.

In [4]:
df['is_read'].unique().tolist()

[0, 1]

In [5]:
# Selecionando apenas as instâncias correspondentes a usuários que leram o livro respectivo
df = df[df['is_read'] == 1]

In [6]:
# Verificando quantas instâncias ainda estão presentes no dataframe
print(f"Total de linhas: {df.shape[0]}")

Total de linhas: 2008206


### Verificando valores duplicados

Como o objetivo futuro será criar um modelo para prever a avaliação de um usuário para um determinado livro, precisamos verificar se existe alguma linha com a combinação de `user_id` e `book_id` duplicados.

In [7]:
df.duplicated(subset=['user_id', 'book_id']).sum()

np.int64(0)

Não há nenhuma instância duplicada, por isso não é necessário nenhum tratamento.

### Dividindo o dataset em Treino, Validação e Teste

Antes de realizar o tratamento de valores ausentes em um dataset, é fundamental dividir os conjuntos de treino, validação e teste. Isso é importante porque tratar valores ausentes antes da divisão pode introduzir vazamento de dados, o que ocorre quando informações do conjunto de teste influenciam os dados de treino, resultando em uma avaliação enviesada do modelo que será treinado no futuro.

In [8]:
def split_data(df : pd.DataFrame, train_frac: float = 0.5, val_frac: float = 0.25, 
               test_frac: float = 0.25) -> list[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Split a DataFrame into training, validation, and test sets.

    Parameters:
    df: The DataFrame to be split.
    train_frac: The fraction of the data to include in the training set.
    val_frac: The fraction of the data to include in the validation set.
    test_frac: The fraction of the data to include in the test set.

    Returns:
    A list containing three DataFrames: the training set, the validation set, and the test set.
    """

    assert train_frac + val_frac + test_frac == 1

    df = df.sample(frac=1, replace=False).reset_index(drop=True)

    qtd_lines = df.shape[0]

    train = df.iloc[:int(qtd_lines * train_frac)]
    validation = df.iloc[int(qtd_lines * train_frac) : int(qtd_lines * (1-test_frac))]
    test = df.iloc[int(qtd_lines * (1-test_frac)):]

    return train, validation, test

In [9]:
df_train, df_val, df_test = split_data(df, train_frac=0.6, val_frac=0.2, test_frac=0.2)


### Verificando valores ausentes

In [11]:
# Verificando quais colunas possuem valores nulos
nan_columns = df_train.isnull().sum()
nan_columns = nan_columns[nan_columns > 0]

print(nan_columns)

book_text_reviews_count         1
language_code              574227
is_ebook                        1
book_rating                     1
book_format                252175
author_id                       5
num_pages                  208777
publication_year           234113
book_ratings_count              1
book_genre                  33287
author_rating                   5
author_reviews_count            5
author_ratings_count            5
dtype: int64


Analisando as colunas que possuem valores nulos é perceptível que as colunas `language_code`, `book_format`, `num_pages`, `publication_year` e `book_genre` possuem uma grande quantidade de valores nulos, por isso devem ser tratadas de forma diferente das demais colunas.

Para tratar as colunas com valores numéricos (`num_pages` e `publication_year`) é possível utilizar a média de todos os valores **do conjunto de treino** (para evitar vazamento de dados) dessa coluna, no entanto para as colunas com valores categóricos é necessário fazer o tratamento de forma diferente.

In [12]:
# Substituindo os valores nulos das colunas numéricas pela média dos demais valores
df_train['num_pages'] = df_train['num_pages'].fillna(df_train['num_pages'].mean()).astype('int64')
df_val['num_pages'] = df_val['num_pages'].fillna(df_train['num_pages'].mean()).astype('int64')
df_test['num_pages'] = df_test['num_pages'].fillna(df_train['num_pages'].mean()).astype('int64')

df_train['publication_year'] = df_train['publication_year'].fillna(df_train['publication_year'].mean()).astype('int64')
df_val['publication_year'] = df_val['publication_year'].fillna(df_train['publication_year'].mean()).astype('int64')
df_test['publication_year'] = df_test['publication_year'].fillna(df_train['publication_year'].mean()).astype('int64')

In [13]:
# Verificando a quantidade de valores distintos nas colunas categóricas
print(f"language_code: {df_train['language_code'].nunique(dropna=False)}")
print(f"book_format: {df_train['book_format'].nunique(dropna=False)}")
print(f"book_genre: {df_train['book_genre'].nunique(dropna=False)}")

language_code: 56
book_format: 113
book_genre: 11


Como a quantidade de valores distintos em cada coluna é muito grande, e será necessário fazer um tratamento a respeito disso no futuro, vamos criar um novo valor para cada coluna referente a "outros", no qual vamos colocar os valores nulos.

In [14]:
# Verificando e imprimindo as linhas que possuem valores NaN em qualquer coluna
rows_with_nan = df[df.isnull().any(axis=1)]

In [15]:
df_train['language_code'] = df_train['language_code'].fillna('other')
df_train['book_format'] = df_train['book_format'].fillna('other')
df_train['book_genre'] = df_train['book_genre'].fillna('other')

df_val['language_code'] = df_val['language_code'].fillna('other')
df_val['book_format'] = df_val['book_format'].fillna('other')
df_val['book_genre'] = df_val['book_genre'].fillna('other')

df_test['language_code'] = df_test['language_code'].fillna('other')
df_test['book_format'] = df_test['book_format'].fillna('other')
df_test['book_genre'] = df_test['book_genre'].fillna('other')

Agora que os valores nulos das linhas mais significativas foram tratados, vamos tratar os demais valores nulos.

In [17]:
# Verificando todas as linhas que ainda possuem valores nulos
df_train[df_train.isnull().any(axis=1)]

,book_text_reviews_count,language_code,is_ebook,book_rating,book_format,author_id,num_pages,publication_year,book_id,book_ratings_count,book_title,book_genre,author_rating,author_reviews_count,author_ratings_count,user_id,is_read,rating,is_reviewed
47328,2.0,other,False,2.91,other,NaN,301,1999,711979,11.0,تهران شهر بی آسمان,fiction,NaN,NaN,NaN,47989,1,4,1
495929,2.0,other,False,2.91,other,NaN,301,1999,711979,11.0,تهران شهر بی آسمان,fiction,NaN,NaN,NaN,46763,1,5,0
537966,NaN,other,NaN,NaN,other,NaN,301,1999,1473309,NaN,دشمن عزیز,other,NaN,NaN,NaN,52456,1,0,1
723513,2.0,other,False,2.91,other,NaN,301,1999,711979,11.0,تهران شهر بی آسمان,fiction,NaN,NaN,NaN,16890,1,2,1
1007264,2.0,other,False,2.91,other,NaN,301,1999,711979,11.0,تهران شهر بی آسمان,fiction,NaN,NaN,NaN,8869,1,3,0


Pode-se perceber que apenas 2 livros distintos são responsáveis por todas essas instâncias com valores nulos, portanto podemos remover elas sem grandes prejuízos.

In [18]:
df_train = df_train.dropna()

In [19]:
# Verificando quais colunas possuem valores nulos
nan_columns = df_train.isnull().sum()
nan_columns = nan_columns[nan_columns > 0]

print(nan_columns)

Series([], dtype: int64)


### Salvando os conjuntos de Treino, Validação e Teste

In [24]:
df_train.to_csv('clean_data/train.csv', index=False, sep=';', encoding='utf-8', header=True)
df_val.to_csv('clean_data/val.csv', index=False, sep=';', encoding='utf-8', header=True)
df_test.to_csv('clean_data/test.csv', index=False, sep=';', encoding='utf-8', header=True)